## Welcome to the FAST tutorial. This guide will provide instructions on how to install and run FAST. The guide assumes you already have an equilibrated structure ready for simulation.

## Obtaining the code

--I recommend making a directory like /Users/bowman/programs and changing into that directory.

--Next, download FAST with the following command

**git clone https://github.com/bowman-lab/fast.git**

--Add the FAST path to your bash profile, in my ~/.bashrc file I have:

**export PYTHONPATH="/project/bowmore/mdward/programs/:$PYTHONPATH"**

## Dependencies

-FAST depends on the following packages -- so make sure you have them installed

    --numpy
    --scipy
    --mdtraj (install instructions at https://mdtraj.org/1.9.3/installation.html)
    --enspara (install instructions at https://enspara.readthedocs.io/en/latest/installation.html)

## Running the code

--Ultimately, you will run a FAST python script at the command line (i.e. **python example-fast-script.py**). Let's take a look at an example script that is setup to run FAST where the geometric component is the distance between two sets of atoms in a protein. As configured, this will likely not run on your setup because your queuing system may be different than ours AND you may want to use a different ranking function / geometric component than distance. We will follow up by exploring these a bit further.

In [ ]:
import glob
import mdtraj as md
import numpy as np
from enspara.cluster import KCenters
from enspara.msm import MSM
from fast import AdaptiveSampling
from fast.md_gen.gromax import Gromax, GromaxProcessing
from fast.msm_gen import ClusterWrap
from fast import DistWrap, PocketWrap
from fast.sampling import rankings, scalings
from fast.submissions.lsf_subs import LSFSub
from fast import SaveWrap
from fast.analysis.pockets import ResiduePockets

def entry_point():


    ###########################################
    #            define parameters            #
    ###########################################

    # SIMULATION PARAMETERS
    
    input_dir = "/path/to/dir/with/input_files"
    # input_dir should be a path to a directory that contains
    # a topol.top file from equilibration
    # an mdp file that contains the simulation parameters
    # a pdb file of the protein with hydrogens (i.e. equilibrated starting structure - no solvent)
    # a start.gro file of the equilibrated system
    
    q_name = "bowman"
    # q_name -- The name of the queue you are going to submit to 
    # currently we have support for LSF and SLURM queues, as well as submitting on a laptop/desktop.
    
    gpu_info = '"num=1:gmodel=TeslaP100_PCIE_16GB"'
    # Name of the GPU, which is needed on our queuing system to select a GPU specific node 
    # This is important for the code to be able to generate the appropriate header for 
    # submitting to your queing system
    # For example, this will end up writing out #BSUB -gpu "num=1:gmodel=TeslaP100_PCIE_16GB"
    
    n_cpus_gromacs = 24
    n_gpus_gromacs = 1
    
    sim_name = "SARS1_nsp16_distance"
    # Name to give the job on your queueing system 
    
    top_filename = "%s/topol.top" % input_dir
    mdp_filename = "%s/npt.mdp" % input_dir
    
    gromacs_source_file = "/project/bowmore/mizimmer/installations/gromacs_2020.1_p100_build"
    #Path to gromacs installation
    
    itp_files = None #glob.glob("./input_files/SARS-2_nsp16/*.itp")

    pbc = 'mol'
    ur = 'compact'
    #Options that will be used by gromacs' trjconv to align trajectories between rounds of FAST

    # CLUSTERING PARAMETERS
    cluster_radius = 0.12
    # Radius to be used for KCenters clustering
    prot_masses = "%s/prot_masses.pdb" % input_dir
    
    atom_indices = "./atom_indices.dat"
    # The atom indices (with respect to prot_masses) you want to use to cluster between rounds of FAST
    # Backbone atoms are a solid default choice.
    
    n_cpus_clustering = 128
    
    # save states
    save_routine = 'full'
    # The type of states to save. Three options: 1) 'masses' saves
    #    only in the centers_masses, 2) 'restarts' saves only the
    #    restarts, and 3) 'full' saves both.
    
    
    save_centers = 'all'
    # The indicator for the set of centers to save. Four options:
    #    1) 'all' will save every center, 2) 'none' will not save any centers,
    #    3) 'restarts' will only save the centers to use for
    #    restarting simulations, and 4) 'auto' will only save new states
    #    that were discovered in previous round of sampling.
    
    save_xtc_centers = True
    # Option to save all the cluster centers as an xtc file.
    
    n_cpus_save = 128

    
    # ANALYSIS PARAMETERS
    
    # Current example is using distance as geometric component
    # RMSD, contacts, potential energy, and pockets are all supported 
    # and can be found in fast/analysis

    atom_pair_filename = "/some/path/to/atom_pairs.dat"
    # This file should have two columns. If we want to measure the distance 
    # between atom 15 and atom 27 (with respect to prot_masses) column 1 would contain
    # a 15 and column 2 would contain a 27. Each row we add contains another pair of atoms
    # that we use to measure a distance. The average distance taken from all the pairs (rows)
    # is used as the FAST geometric component.
    
    p_norm = 1
    # The p-norm to use when processing distance pairs. i.e.
    #    ||x||_p := sum(|x_i|^p)^(1/p)
    # A value of 1 just takes the mean.
    
    set_points = None
    # A list of reference distances for each atom pair. If provided,
    # reports deviation from these distances.
    
    center_of_mass = False
    # calculates the distance between
    # the center of mass of the first column of atoms and the center
    # of mass of the second column of atoms.

    # RANKING PARAMETERS
    directed_scaling = scalings.feature_scale(maximize=True)
    # Scales geometric component (e.g. distance) and statistical component (e.g. counts)
    # to values between between 0 and 1
    # Feature scales data: (x - xmin) / (xmax - xmin)
    # With maximize=True, FAST looks to maximize the distance
    # if maximize=False, FAST looks to minimize the distance
    
    distance_metric = md.rmsd
    # This will ultimately be used to discourage FAST from choosing geometrically similar 
    # states when choosing states for further simulation.
    width = 0.36
    # This is the gaussian spread that is used to distinguish between states. 

    
    # ADAPTIVE SAMPLING PARAMETERS
    
    starting_structure = "%s/start.gro" % input_dir
    submission_obj = LSFSub(
        'bowman', n_cpus=128, R="model=AMDEPYC_7742")
    # This creates a script to submit clustering and analysis jobs to the appropriate
    # nodes in your queue with the appropriate keyword arguments.
    # This may require customization for your specific queuing system. Please 
    # explore submissions/lsf_subs.py, submissions/os_sub.py, submissions/slurm_subs.py
    # for examples on how to build a class that is specific to your queuing system.
    
    n_gens = 15
    #Number of rounds or "generations" of FAST to run
    n_kids = 10
    #Number of simulations to run in each generation of FAST
    
    update_freq = 1
    #The number of generations between a full reclustering of states and
    #    analysis of cluster centers. Defaults to never reclustering
    #    (continually adds new cluster centers without changing previously
    #    discovered centers).
    
    continue_prev = False
    # If you are continuing a FAST run, you can set this to True
    
    output_dir = "FASTDistance-SARS1_nsp16"
    # Name of directory to write to store FAST output

    ############################################
    #            initialize objects            #
    ############################################

    # SIMULATIONS OBJECTS
    gro_submission = LSFSub(
        q_name, n_tasks=n_cpus_gromacs, job_name=sim_name, gpu=gpu_info, R='"span[hosts=1]"')
    
    gro_processing = GromaxProcessing(
        align_group=10, output_group=10, pbc=pbc, ur=ur)
    # In this specific example, 10 is the index for protein. Check your
    # .ndx file and select the index that chooses protein.
    
    sim_obj = Gromax(
        top_file=top_filename, mdp_file=mdp_filename, n_cpus=n_cpus_gromacs,
        n_gpus=n_gpus_gromacs, processing_obj=gro_processing,
        submission_obj=gro_submission, pin='on',
        source_file=gromacs_source_file, itp_files=itp_files)

    # CLUSTERING OBJECT
    base_clust_obj = KCenters(metric=md.rmsd, cluster_radius=cluster_radius)
    clust_obj = ClusterWrap(
        base_struct=prot_masses, base_clust_obj=base_clust_obj,
        atom_indices=atom_indices, n_procs=n_cpus_clustering)

    # SAVE STATE OBJECT
    save_state_obj = SaveWrap(
        save_routine=save_routine, centers=save_centers,
        n_procs=n_cpus_save, save_xtc_centers=save_xtc_centers)

    # ANALYSIS OBJECT
    anal_obj = DistWrap(
        atom_pairs=atom_pair_filename, p_norm=p_norm, set_points=set_points,
        center_of_mass=center_of_mass)

    # RANKING OBJECT
    ranking_obj = rankings.FAST(
        directed_scaling=directed_scaling, distance_metric=distance_metric,
        width=width)

    # SAMPLING OBJECT
    a = AdaptiveSampling(
        starting_structure, n_gens=n_gens, n_kids=n_kids, sim_obj=sim_obj,
        cluster_obj=clust_obj, save_state_obj=save_state_obj,
        analysis_obj=anal_obj, ranking_obj=ranking_obj,
        continue_prev=continue_prev, update_freq=update_freq,
        sub_obj=submission_obj, output_dir=output_dir)

    ##############################################
    #                run sampling                #
    ##############################################

    # run
    a.run()


if __name__=='__main__':
    entry_point()

-- Given the above script (lets call it "example-fast-script.py") if everything was configured properly for your system, we would now simply run:

**python example-fast-script.py**

--In practice, I am used to running this on a cluster with an LSF queueing system. I CD into a directory where I want the FAST output to be, then I use **screen** or **tmux** before submitting that python command on the head node. The FAST code handles submitting jobs to the appropriate high-compute nodes (e.g. submitting to a GPU node for running the simulations, then maybe submitting to a CPU node for clustering and ranking states for further simulation)

## Adapting the above script to your needs

--It is very likely that your queueing system is different than ours. You will at the very least need to know how the queue submission classes work in order to configure your script properly. In some cases, you may need to build your own class. We will cover some of this below

--It is very likely that you will want a different ranking function than the distance example used above. Here will explore how to setup the other order parameters that we already have supported. You can always create your own class if you would like a new order parameter :D



### Configuring for your queueing system

--You will need to create two objects that interface with your queuing system (as we did with **submission_obj** and **gro_submission** in the above code)

--In this case, **submission_obj** is used to submit jobs to a compute node to separately do clustering, building MSMs, ranking states, and saving states. Similarly, **gro_submission** is used to submit a job to a compute node to run the Gromacs simulations. 

--Under the hood, these objects are simply creating a file (default name is lsf_submission) that has the format required by the queueing system. For example, for the first generation of clustering, **submission_obj** creates a file *clusterer_submission_gen000* that looks like this:

In [ ]:
#!/bin/bash

# specify resources
#BSUB -n 1

# max wallclock time
#BSUB -ptl 1500:00

# queue
#BSUB -q bowman

# name and output
#BSUB -J LSF_Sub
#BSUB -o lsf_output-%J.log
#BSUB -e lsf_output-%J.log

# additional specs
#BSUB -n_cpus 128
#BSUB -R "model=AMDEPYC_7742"

sync
python clusterer.py

-- then, it submits this with **bsub < clusterer_submission_gen000**

--Recall that this generated file comes from

submission_obj = LSFSub(
        'bowman', n_cpus=128, R="model=AMDEPYC_7742")
        
--**LSFSub** requires that you provide the queue name (bowman) and then you can provide optional keyword arguments that were necessary in our case (e.g. -R "model=AMDEPYC_7742"). You may have some of these optional parameters that we can't know ahead of time, but have provided a way for you to add them in (just add them as keyword arguments). The key here is to just make sure your *submission_obj* contains the necessary parameters to generate the header of your submission file.

--This example demonstrates how to use the **LSFSub** class, but the same ideas apply if you use the **SlurmSub** class. If you don't use LSF or SLURM, please create your own class and look at submissions/lsf_subs.py as an example.

--Side note:
    You might wonder where **python clusterer.py** comes from. This gets autogenerated and in this case just calls the run method of cluster_obj you defined before (i.e. it performs the clustering).

--Our other object to interface with the queuing system is **gro_submission**, which looks like this (from above)

gro_submission = LSFSub(
        q_name, n_tasks=n_cpus_gromacs, job_name=sim_name, gpu=gpu_info, R='"span[hosts=1]"')
        
--This gets passed in as a parameter to make a **Gromax** object which will ultimately generate the following:

In [ ]:
#!/bin/bash

# specify resources
#BSUB -n 24

# max wallclock time
#BSUB -ptl 1500:00

# queue
#BSUB -q bowman

# name and output
#BSUB -J SARS1_nsp16_distance
#BSUB -o lsf_output-%J.log
#BSUB -e lsf_output-%J.log

# additional specs
#BSUB -gpu "num=1:gmodel=TeslaP100_PCIE_16GB"
#BSUB -R "span[hosts=1]"

source /project/bowmore/mizimmer/installations/gromacs_2020.1_p100_build

if [ ! -f "/path/to/FAST/output/gen0/kid0/md.tpr" ]; then
    echo "Didn't find md.tpr, running grompp..."
    ls
    pwd
    gmx grompp -f /path/to/npt.mdp -c /path/to/FAST/output/gen0/kid0/start.gro -p /path/to/topol.top -o md -maxwarn 2
else
    echo "Found md.tpr, not running grompp"
fi

gmx mdrun -cpi state -g md -s md -o md -c after_md -v -nt 24 -x frame0 -ntmpi 1 -ntomp 24 -pin on
echo '10 0' | gmx trjconv -f frame0.xtc -o frame0_aligned.xtc -s md.tpr -center -pbc mol -ur compact
echo '10 10' | gmx trjconv -f frame0.xtc -o frame0_masses.xtc -s md.tpr -center -pbc mol -ur compact

If you do not use SLURM or LSF, you will have to configure a class for your own queueing system. Please look at /fast/submissions/lsf_sub.py -- this can serve as a template for building your own custom class.

### Configuring for the geometric component (order parameter) you want to use

The example FAST python script shown above was configured for setting the geometric component to measure the distance between two groups of atoms. Here, we will demonstrate some other examples, but you should peruse through the python files in fast/analysis/ to see how to configure FAST with other geometric components. We have already created classes that allow you to configure the geometric component for the following:

-fraction of native contacts

-rmsd 

-potential energy

-pocket volume

Let's review how we configured FAST to maximize a distance between a group of atoms by highlighting the relevant parts from the python script above:

In [ ]:
# ANALYSIS PARAMETERS

# Current example is using distance as geometric component
# RMSD, contacts, potential energy, and pockets are all supported 
# and can be found in fast/analysis

atom_pair_filename = "/some/path/to/atom_pairs.dat"
# This file should have two columns. If we want to measure the distance 
# between atom 15 and atom 27 (with respect to prot_masses) column 1 would contain
# a 15 and column 2 would contain a 27. Each row we add contains another pair of atoms
# that we use to measure a distance. The average distance taken from all the pairs (rows)
# is used as the FAST geometric component.

p_norm = 1
# The p-norm to use when processing distance pairs. i.e.
#    ||x||_p := sum(|x_i|^p)^(1/p)
# A value of 1 just takes the mean.

set_points = None
# A list of reference distances for each atom pair. If provided,
# reports deviation from these distances.

center_of_mass = False
# calculates the distance between
# the center of mass of the first column of atoms and the center
# of mass of the second column of atoms.

#ANALYSIS OBJECT
anal_obj = DistWrap(
    atom_pairs=atom_pair_filename, p_norm=p_norm, set_points=set_points,
    center_of_mass=center_of_mass)

### RMSD

Let's say we want to use RMSD to a known structure as the geometric component instead of distance between atoms as the geometric component. We should replace the above code with the following:

In the first example, we will use RMSD as the geometric component to encourage the simulation to explore away from the starting structure

In [ ]:
#ANALYSIS PARAMETERS
# none

# RANKING PARAMETERS
directed_scaling = scalings.feature_scale(maximize=True)
#Since we are exploring conformational space from a starting structure,
# We want to maximize the RMSD relative to the starting structure, so maximize=True

#ANALYSIS OBJECT
anal_obj = RMSDWrap(prot_masses)
#All we need to do is supply the starting structure, which we previously stored as the variable prot_masses


In the next example, we will use RMSD so that FAST helps us hone in on a target structure

In [ ]:
#ANALYSIS PARAMETERS
target_structure = md.load("path/to/target/structure.pdb")

# RANKING PARAMETERS
directed_scaling = scalings.feature_scale(maximize=False)
# Since we are honing in on a target structure,
# We want to minimize the RMSD to the target structure, so minimize=True

#ANALYSIS OBJECT
anal_obj = RMSDWrap(target_structure)

### Pockets

Here we will show you how to configure the geometric component to maximize pocket volume

In [ ]:
# ANALYSIS PARAMETERS

min_rank = 6
# Minimum rank for defining a pocket element. Ranges from 1-7, 1
# being very shallow and 7 being a fully enclosed pocket element.

probe_radius = 0.14
# The radius of the grid point to probe for pocket elements.

min_cluster_size = 3
# The minimum number of adjacent pocket elements to consider a
# true pocket. Trims pockets smaller than this size.

n_cpus = 24 

anal_obj = PocketWrap(
        min_rank=min_rank, min_cluster_size=min_cluster_size, n_cpus=n_cpus,
        probe_radius=probe_radius)

### Custom geometric component

Feel free to create a class of your own, like **AngleWrap**, which might maximize the angle between two protein domains. Please follow the conventions used in the python scripts in fast/analysis/